In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

In [ ]:
device = 'cpu'

# Preparing data

In [ ]:
#function to prepare shifted data for RNN

from copy import deepcopy

def prepare_shifted_df(df, n_steps, input_name):
    df = deepcopy(df)
    
    # df.set_index('Month', inplace=True)
    
    for i in reversed(range(1, n_steps + 1)):
        new_name = input_name + f"(t-{i})"
        df[new_name] = df.iloc[:,0].shift(i)
        
    df.dropna(inplace=True)
    
    return df.iloc[:,1:]

lookback = 36

In [ ]:
#prepare shifted data of inputs and output for RNN

df = pd.read_csv('../Data/Employment_pch_1960_no_covid.csv')
df = df.iloc[:,1:]
df.set_index('Month', inplace=True)
# print(df)
shifted_df0 = prepare_shifted_df(df[['Total_private_pch']], lookback, 'Total_private_pch')

df = pd.read_csv('../Data/Input_features_1960_no_covid.csv')
df = df.iloc[:,1:]
df.set_index('Month', inplace=True)
# print(df)
shifted_df1 = prepare_shifted_df(df[['Fedfund_rate']], lookback, 'Fedfund_rate')
shifted_df2 = prepare_shifted_df(df[['GDP_pch']], lookback, 'GDP_pch')
shifted_df3 = prepare_shifted_df(df[['CPI_pc1']], lookback, 'CPI_pc1')
shifted_df4 = prepare_shifted_df(df[['Loan_pch']], lookback, 'Loan_pch')
shifted_df5 = prepare_shifted_df(df[['House_ch']], lookback, 'House_ch')

In [ ]:
#function to scale data and convert to numpy, easier to manipulate into tensor

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

def scale_df(df, scaler):
    df_np = df.to_numpy()
    return scaler.fit_transform(df_np)

In [ ]:
# making input data the right dimension in ndarray and visualized scaled inputs and output

X0 = scale_df(shifted_df0, scaler)
X1 = scale_df(shifted_df1, scaler)
X2= scale_df(shifted_df2, scaler)
X3 = scale_df(shifted_df3, scaler)
X4 = scale_df(shifted_df4, scaler)
X5 = scale_df(shifted_df5, scaler)

temp_list = [X0, X1, X2, X3, X4, X5]
X = np.zeros([X0.shape[0], X0.shape[1], 6])

for i in range(6):
    X[:,:,i] = temp_list[i]
    
print(f"X shape is {X.shape}")
plt.plot(X0[:,0], label = 'Total private employment')
plt.plot(X1[:,0], label = 'Fed funds rate')
plt.plot(X2[:,0], label = 'GDP')
plt.plot(X3[:,0], label = 'CPI')
plt.plot(X4[:,0], label = 'Loan',  linestyle = 'dashed', color = 'gray')
plt.plot(X5[:,0], label = 'House')

plt.legend()
plt.show()

np.isnan(X).any(0).any(0)

In [ ]:
y_df = pd.read_csv('../Data/Employment_pch_1960_no_covid.csv')

y_df = y_df[['Month', 'Total_private_pch']]
y_df = y_df.iloc[lookback:,:]
y_df.set_index('Month', inplace = True)
# print(y_df)
y = y_df.to_numpy()
y_dir = np.sign(y)
y_dir[y_dir == -1] = 0
y_dir = y_dir[:,0]

X.shape, y_dir.shape

**Train test split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_dir, test_size=0.3, shuffle = False, random_state=8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
y_test = torch.tensor(y_test, dtype = torch.long)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Prepare custom dataset and dataloader for Pytorch**

In [ ]:

from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)
train_dataset

In [ ]:
#wrap dataset in data loaders to get the batches
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.dtype, y_batch.dtype)
    print(x_batch.shape, y_batch.shape)
    break

# Define rnn class, train loop and test loop functions

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, nonlinearity='tanh', batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  
        # or:
        #out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.softmax(out)
        return out
    

In [ ]:
def train_one_epoch(dataloader, model, loss_function, optimizer):
    model.train()
    running_loss = 0.0
    
    for batch_index, batch in enumerate(dataloader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [ ]:
def validate_one_epoch(dataloader, model, loss_function):
    model.eval() #evaluation mode
    running_loss = 0.0
    
    for batch_index, batch in enumerate(dataloader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad(): #not calculating gradients because we're not updating model
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

**Define classification error**

In [ ]:
def error_rate(y_predict, y_actual):
#     assert(len(y_predict) == len(y_actual))
    max_ind_train = torch.argmax(y_predict, axis = 1)
    error = torch.sum(abs(y_actual - max_ind_train)) / len(max_ind_train)
    return error

# Training RNN

**Initialize the RNN model with parameters**

In [ ]:
input_size = 6
hidden_size = 64
num_layers = 6
num_classes = 2

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
model

In [ ]:
learning_rate = 0.005
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

all_train_error = []
all_test_error = []

num_epochs = 500
best_error = 10.0
best_model_state = {}
for epoch in range(num_epochs):
    
    train_one_epoch(train_loader,model,loss_function,optimizer)
    validate_one_epoch(test_loader, model, loss_function)

    with torch.no_grad():
        # predicted_train = model(X_train.to(device)).to('cpu')
        predicted_train = model(X_train)
        train_error = error_rate(predicted_train, y_train)
        all_train_error.append(train_error)

    with torch.no_grad():
        # predicted_test = model(X_test.to(device)).to('cpu')
        predicted_test = model(X_test)
        test_error = error_rate(predicted_test, y_test)
        all_test_error.append(test_error)

    if epoch % 20 == 19:
        print(f'Epoch: {epoch + 1}')
        print(f"training data error is {train_error}")
        print(f"test data error is {test_error}")
    if test_error < best_error:
        best_model_state = deepcopy(model.state_dict())
        best_error = test_error


**Print best model in the epochs**

In [ ]:
best_model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
best_model.load_state_dict(best_model_state)
best_model.eval()

**Plot and print test and training errors**

In [ ]:
with torch.no_grad():
    predicted_train = best_model(X_train.to(device)).to('cpu')
train_error = error_rate(predicted_train, y_train)
print(f"training classification data error is {train_error}")

with torch.no_grad():
    predicted_test = best_model(X_test.to(device)).to('cpu')
test_error = error_rate(predicted_test, y_test)
print(f"test classification data error is {test_error}")

In [ ]:
plt.plot(all_train_error, label = 'Train error')
plt.plot(all_test_error, label = 'Test error')
plt.legend()
plt.show()

In [ ]:
# print("Model's state_dict:")
# for param_tensor in best_model.state_dict():
#     print(param_tensor, "\t", best_model.state_dict()[param_tensor].size())

In [ ]:
#save model
# torch.save(best_model_state, './sw_rnn_classify_construction_savedmodel.pt')


# model_scripted = torch.jit.script(model) # Export to TorchScript
# model_scripted.save('model_scripted.pt') # Save

In [ ]:
#load model
# model2 = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
# model2.load_state_dict(torch.load('./sw_rnn_classify_construction_savedmodel.pt'))
# model2.eval()

In [ ]:
# predicted_test2 = model2(X_test)
# test_error2 = error_rate(predicted_test2, y_test)
# print(f"Saved model produces test data error =  {test_error2}")